# Model Training

### Import Libraries & Load Metadata

In [91]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pathlib import Path
import cv2
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import tensorflow.keras.backend as K

In [92]:
meta_path = "../data/cibs-ddsm/metadata/meta.csv"
df = pd.read_csv(meta_path)

### Load Images into Memory

In [93]:
def load_images(df, size=(224, 224)):
    X, y = [], []
    for _, row in df.iterrows():
        img = cv2.imread(row["processed_path"], cv2.IMREAD_GRAYSCALE)
        if img is not None:
            X.append(img)
            y.append(row["label"])
    X = np.array(X)[..., np.newaxis] / 255.0  
    y = np.array(y)
    return X, y

In [94]:
y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

### Split dataset

In [95]:
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
X_train, y_train = load_images(train_df)
X_test, y_test = load_images(test_df)

### Build CNN Model with Augmentation

### Batch normalisation 

In [96]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Flatten,
                                     Dense, Dropout, RandomFlip,
                                     RandomRotation, RandomZoom)

model = Sequential([
    Input(shape=(224, 224, 1)), 
    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.1),
    Conv2D(32, (3, 3), activation="relu"),
    MaxPooling2D(),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

In [97]:
def focal_loss(alpha=0.25, gamma=2.0):
    def focal_loss_fixed(y_true, y_pred):
        eps = K.epsilon()
        y_pred = K.clip(y_pred, eps, 1. - eps)
        pt = tf.where(K.equal(y_true, 1), y_pred, 1 - y_pred)
        return -K.mean(alpha * K.pow(1. - pt, gamma) * K.log(pt))
    return focal_loss_fixed

In [98]:
from tensorflow.keras.metrics import AUC

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

## Check for label distribution

In [99]:
import pandas as pd
df = pd.read_csv("../data/cibs-ddsm/metadata/labels_resolved.csv")
df["label"].value_counts()

label
0    2111
1    1457
Name: count, dtype: int64

In [100]:
unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))

{0: 1689, 1: 1165}


## Calculate Class Weights

In [101]:
from sklearn.utils.class_weight import compute_class_weight

class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train.flatten()),
    y=y_train.flatten()
)
class_weights = dict(enumerate(class_weights_array))

### Add early stopping

In [102]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_auc', patience=5, restore_best_weights=True, mode='max')

In [103]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)

In [104]:
y_train = y_train.ravel()
y_val = y_val.ravel()
y_test = y_test.ravel()

In [105]:
print(X_train.shape, y_train.shape)

(2283, 224, 224, 1) (2283,)


In [106]:
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    class_weight=class_weights,
    callbacks=[early_stop]
)

Epoch 1/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 17s 227ms/step - accuracy: 0.4489 - auc: 0.4742 - loss: 1.0748 - val_accuracy: 0.5919 - val_auc: 0.4852 - val_loss: 0.6841
Epoch 2/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 16s 223ms/step - accuracy: 0.5517 - auc: 0.4889 - loss: 0.6960 - val_accuracy: 0.4448 - val_auc: 0.5093 - val_loss: 0.6933
Epoch 3/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 16s 223ms/step - accuracy: 0.4255 - auc: 0.5062 - loss: 0.6921 - val_accuracy: 0.4431 - val_auc: 0.4988 - val_loss: 0.6932
Epoch 4/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 16s 222ms/step - accuracy: 0.4604 - auc: 0.4785 - loss: 0.6919 - val_accuracy: 0.4186 - val_auc: 0.4957 - val_loss: 0.6935
Epoch 5/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 16s 221ms/step - accuracy: 0.4182 - auc: 0.4984 - loss: 0.6930 - val_accuracy: 0.5061 - val_auc: 0.4839 - val_loss: 0.6930
Epoch 6/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 16s 223ms/step - accuracy: 0.4941 - auc: 0.5035 - loss: 0.6939 - val_accuracy: 0.4641 - val_auc: 0.4722 - val_loss: 0.6930
Epoch 7/30
72/72 ━━━━━━━━━━━━━━━━━

In [107]:
results = model.evaluate(X_test, y_test, return_dict=True)
print(f"Test Loss: {results['loss']:.4f}")
print(f"Test Accuracy: {results['accuracy']:.4f}")
print(f"Test AUC: {results['auc']:.4f}")
model.save("../results/model_weights/final_model.h5")

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.4513 - auc: 0.4810 - loss: 0.6935


Test Loss: 0.6932
Test Accuracy: 0.4776
Test AUC: 0.4993


In [108]:
X_test_rgb = np.repeat(X_test, 3, axis=-1)

In [112]:
from tensorflow.keras.metrics import AUC

metric_auc = AUC()
metric_auc.update_state(y_test, y_probs)
print(f"Keras AUC (manual): {metric_auc.result().numpy():.4f}")
print(f"sklearn AUC: {roc_auc_score(y_test, y_probs):.4f}")

Keras AUC (manual): 0.5016
sklearn AUC: 0.4959


In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

# Predict probabilities
y_probs = model.predict(X_test)

# Convert probabilities to binary predictions
y_pred = (y_probs > 0.5).astype(int)

# Print classification report
print(classification_report(y_test, y_pred, target_names=["Benign", "Malignant"]))

# Print AUC score
auc = roc_auc_score(y_test, y_probs)
print(f"ROC AUC Score: {auc:.4f}")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

y_pred_probs = model.predict(X_test).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

# Confusion Matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label="ROC Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.show()

print("ROC AUC Score:", roc_auc_score(y_test, y_pred_probs))

In [ ]:
np.save("../data/cibs-ddsm/processed/X_test.npy", X_test)
np.save("../data/cibs-ddsm/processed/y_test.npy", y_test)